In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

In [2]:
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from  src.database.dbconnection import getconnection
from python_code.transform import DataTransform
from src.model.models import CandidatesProcessed
from src.model.models import Countries
from sqlalchemy.exc import SQLAlchemyError



In [3]:
engine = getconnection()
Session = sessionmaker(bind=engine)
session = Session()

Conected successfully to database workshop1!


In [4]:
try:
    if inspect(engine).has_table('Countries'):
        Countries.__table__.drop(engine)
    Countries.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [5]:
try:
    if inspect(engine).has_table('CandidatesProcessed'):
        CandidatesProcessed.__table__.drop(engine)
    CandidatesProcessed.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [7]:
try:
    transform_candidates = DataTransform('../data/candidates.csv')
    transform_candidates.insert_id()
    transform_candidates.HiredOrNotHired()
    transform_candidates.rename()
    transform_candidates.technology_by_category()
    transform_candidates.FullNameReplace()
    transform_candidates.ApplicationDateToDateType()
    
     
    transform_countrys = transform_candidates.NormalizeCountry()
    
    transform_countrys.to_sql('Countries', con=engine, if_exists='append', index=False)
    transform_candidates.df.to_sql('CandidatesProcessed', con=engine, if_exists='append', index=False)
    
    print("Data uploaded")

except SQLAlchemyError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    if hasattr(engine, 'dispose'):
        engine.dispose()
    if 'session' in locals():
        session.close()



Data uploaded
